In [1]:
import pandas as pd
import sqlite3

In [2]:
con = sqlite3.connect('db')

In [3]:
users = pd.DataFrame({'id':[1,2,3], 'name':['Nick', 'Zahir', 'Amin']})
items = pd.DataFrame({'user_id':[1,3,3], 'item_name':['bread', 'milk', 'vine'], 'value':[22, 30, 50]})

In [4]:
users.to_sql('users', con, index=False, if_exists='replace')
items.to_sql('items', con, index=False, if_exists='replace')

3

In [5]:
def select(sql):
    return pd.read_sql(sql, con)

In [6]:
sql = '''select * from users t'''

In [7]:
select(sql)

,id,name
0,1,Nick
1,2,Zahir
2,3,Amin


In [8]:
sql = '''select * from items t'''

In [9]:
select(sql)

,user_id,item_name,value
0,1,bread,22
1,3,milk,30
2,3,vine,50


In [10]:
sql = '''select u.*, i.item_name from users as u
left join items as i on u.id = i.user_id'''

In [11]:
select(sql)

,id,name,item_name
0,1,Nick,bread
1,2,Zahir,None
2,3,Amin,milk
3,3,Amin,vine


In [12]:
users = pd.DataFrame({'id':[1,2,3], 'name':['Nick', 'Zahir', 'Amin'],
                     'victory':[2,3,33]})

In [13]:
users.to_sql('users', con, index=False, if_exists='replace')

3

In [14]:
sql = '''select * from users'''

In [15]:
select(sql)

,id,name,victory
0,1,Nick,2
1,2,Zahir,3
2,3,Amin,33


In [16]:
sql = '''select * from items as i'''

In [17]:
select(sql)

,user_id,item_name,value
0,1,bread,22
1,3,milk,30
2,3,vine,50


In [18]:
sql = '''select u.id, u.name, u.victory,
count(i.item_name) as item_cnt,
coalesce(sum(i.value),0) as value_sum
from users as u
left join items as i on u.id = i.user_id
group by u.id, u.name, u.victory
'''

In [19]:
select(sql)

,id,name,victory,item_cnt,value_sum
0,1,Nick,2,1,22
1,2,Zahir,3,0,0
2,3,Amin,33,2,80


In [20]:
sql = '''
        with item_agg as (
        select t.user_id,
        count(t.item_name) as item_cnt,
        sum(t.value) as value_sum
        from items as t
        group by t.user_id)
        
        select t.id, t.name, t.victory,
        coalesce(i.item_cnt, 0),
        coalesce(i.value_sum, 0)
        from users as t
        left join item_agg as i on user_id == id 
        '''

In [21]:
select(sql)

,id,name,victory,"coalesce(i.item_cnt, 0)","coalesce(i.value_sum, 0)"
0,1,Nick,2,1,22
1,2,Zahir,3,0,0
2,3,Amin,33,2,80


In [22]:
# sql = '''select * from users t, items i
# where t.id = i.user_id''' 

# старая версия join-а сейчас не используются
# еще есть using - это используются места ON но если id есть у обоих таблицах

In [23]:
select(sql)

,id,name,victory,"coalesce(i.item_cnt, 0)","coalesce(i.value_sum, 0)"
0,1,Nick,2,1,22
1,2,Zahir,3,0,0
2,3,Amin,33,2,80


In [24]:
items = pd.DataFrame({'user_id':[1,3,3,4], 'item_name':['bread', 'milk', 'vine', 'fruits'], 'value':[22, 30, 5, 59]})

In [29]:
sql = '''select * from items'''

In [30]:
select(sql)

,user_id,item_name,value
0,1,bread,22
1,3,milk,30
2,3,vine,50


In [31]:
sql = '''select * from users'''

In [32]:
select(sql)

,id,name,victory
0,1,Nick,2
1,2,Zahir,3
2,3,Amin,33


In [41]:
sql = '''select t.*, i.* from users as t
        left join items as i on t.id = i.user_id'''

In [42]:
select(sql)

,id,name,victory,user_id,item_name,value
0,1,Nick,2,1.0,bread,22.0
1,2,Zahir,3,NaN,None,NaN
2,3,Amin,33,3.0,milk,30.0
3,3,Amin,33,3.0,vine,50.0


In [46]:
sql = '''select t.*, i.* from items as t
        left join users as i on t.user_id = i.id '''

In [47]:
select(sql)

,user_id,item_name,value,id,name,victory
0,1,bread,22,1,Nick,2
1,3,milk,30,3,Amin,33
2,3,vine,50,3,Amin,33


In [50]:
sql = '''select * from items as t
        full join users as i on t.user_id = i.id '''

In [51]:
select(sql)

,user_id,item_name,value,id,name,victory
0,1.0,bread,22.0,1,Nick,2
1,3.0,milk,30.0,3,Amin,33
2,3.0,vine,50.0,3,Amin,33
3,NaN,None,NaN,2,Zahir,3


In [53]:
df = pd.read_csv('../german_credit_augmented.csv')

In [55]:
df.to_sql('german_credit', con, index=False, if_exists='replace')

1000

In [58]:
sql = 'select  * from german_credit as t limit 5'

In [59]:
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,None,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,None,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [66]:
sql = '''
       select 1 as user_id 
       union all
       select 2 as user_id
       union all 
       select 3 as user_id
       '''

In [67]:
select(sql)

,user_id
0,1
1,2
2,3


In [76]:
sql = '''
        select date('2023-10-20') as month
        union all
        select date('2024-02-10') as month
        '''

In [77]:
select(sql)

,month
0,2023-10-20
1,2024-02-10


In [91]:
sql ='''with users as (
    select 1 as user_id 
    union all
    select 2 as user_id
    union all 
    select 3 as user_id),

    month as (select date('2023-10-20') as month
    union all
    select date('2024-02-10') as month)

    select * from users as t
    join month as m on 1=1
    '''

In [92]:
select(sql)

,user_id,month
0,1,2023-10-20
1,2,2023-10-20
2,3,2023-10-20
3,1,2024-02-10
4,2,2024-02-10
5,3,2024-02-10


In [93]:
transactions = pd.read_csv('../german_credit_augmented_transactions.csv')

In [94]:
transactions.head(5)

,dt,client_id,amount
0,2008-04-06 11:54:47,950,161.38
1,2007-07-28 00:00:19,418,35.34
2,2008-03-14 20:43:54,131,146.50
3,2007-12-18 13:03:24,353,119.21
4,2007-11-09 05:18:30,849,105.24


In [95]:
transactions.to_sql('transactions', con, index=False, if_exists='replace')

4275

In [145]:
sql = '''select date(t.dt, 'start of month') as mount, 
sum(t.amount) as transactions_amount, 
count(1) as transactions_cnt from transactions as t
    group by 1
    order by 1'''

In [146]:
select(sql)

,mount,transactions_amount,transactions_cnt
0,2007-05-01,450912.77,338
1,2007-06-01,551664.83,379
2,2007-07-01,494134.50,304
3,2007-08-01,426903.23,255
4,2007-10-01,634846.49,332
5,2007-11-01,500420.98,389
6,2007-12-01,561449.89,364
7,2008-01-01,630137.22,413
8,2008-02-01,337043.47,228
9,2008-03-01,425599.09,309
